In [ ]:
def reproduceResult():
  seed_value= 0

  
  with tf.device("/cpu:0"):
    ...


  os.environ['PYTHONHASHSEED']=str(seed_value)
  np.random.seed(0)
  rn.seed(0)


  session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, 
                                          inter_op_parallelism_threads=1)


  tf.compat.v1.set_random_seed(seed_value)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
  tf.compat.v1.keras.backend.set_session(sess)
  tf.compat.v1.keras.backend.clear_session()

  


In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
  
import os 
import numpy as np
import tensorflow as tf
import random as rn
from tensorflow import keras

reproduceResult()
import tempfile
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K


from tensorflow.keras import callbacks

from keras_lr_finder import LRFinder
from clr_callback import CyclicLR

import tensorflow_model_optimization as tfmot



import keras_tuner
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.


In [ ]:
reproduceResult()

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/IMDB/imdb_master.csv',encoding="latin-1")

# df2 = pd.read_csv('imdb_master.csv',encoding="latin-1")
df = df.drop(['Unnamed: 0','type','file'],axis=1)
df.columns = ["review","sentiment"]
df = df[df.sentiment != 'unsup']
df['sentiment'] = df['sentiment'].map({'pos': 1, 'neg': 0})
# df = pd.concat([df1, df2]).reset_index(drop=True)
  
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
  
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
  

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
  
 
def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if  not word in stop_words]
    text = " ".join(text)
    return text
  
df['Processed_Reviews'] = df.review.apply(lambda x: clean_text(x))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
df

,review,sentiment,Processed_Reviews
0,Once again Mr. Costner has dragged out a movie...,0,mr costner ha drag movie far longer necessary ...
1,This is an example of why the majority of acti...,0,example majority action film generic bore real...
2,"First of all I hate those moronic rappers, who...",0,first hate moronic rapper couldnt act gun pres...
3,Not even the Beatles could write songs everyon...,0,even beatles could write song everyone like al...
4,Brass pictures (movies is not a fitting word f...,0,brass picture movie fit word really somewhat b...
...,...,...,...
49995,"Seeing as the vote average was pretty low, and...",1,see vote average wa pretty low fact clerk vide...
49996,"The plot had some wretched, unbelievable twist...",1,plot wretched unbelievable twist however chemi...
49997,I am amazed at how this movie(and most others ...,1,amaze movieand others ha average 5 star lower ...
49998,A Christmas Together actually came before my t...,1,christmas together actually come time ive rais...


In [ ]:
data_95, data_5 = train_test_split(df, test_size=0.05,  random_state = 42)

In [ ]:
data_5 = data_5.sample(frac=1).reset_index(drop=True)


data_5.to_csv(r'/content/drive/MyDrive/Colab Notebooks/Dataset/IMDB/imdb_5_percent.csv', index=True,header=True)

In [ ]:
data_5

,review,sentiment,Processed_Reviews
0,"Although the story is fictional, it draws from...",1,although story fictional draw reality history ...
1,I had numerous problems with this film.<br /><...,0,numerous problem filmbr br contain basic factu...
2,hi for all the people who have seen this wonde...,1,hi people see wonderful movie im sure thet wou...
3,I have just watched the season 2 finale of Doc...,1,watch season 2 finale doctor apart couple dull...
4,Oh dear. Some of the best talent in British TV...,0,oh dear best talent british tv make serial ass...
...,...,...,...
2495,Home Room really surprised me. In comparison t...,1,home room really surprise comparison movie wri...
2496,Too bad somebody did not have the smarts to re...,1,bad somebody smart release movie theater never...
2497,I know this movie is a low-budget horror movie...,0,know movie lowbudget horror movie intend favor...
2498,Still funny after all these years. Midnight Ma...,1,still funny year midnight madness good enterta...


In [ ]:
temp = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/IMDB/imdb_5_percent.csv')

del temp['Unnamed: 0']
temp


,review,sentiment,Processed_Reviews
0,"Although the story is fictional, it draws from...",1,although story fictional draw reality history ...
1,I had numerous problems with this film.<br /><...,0,numerous problem filmbr br contain basic factu...
2,hi for all the people who have seen this wonde...,1,hi people see wonderful movie im sure thet wou...
3,I have just watched the season 2 finale of Doc...,1,watch season 2 finale doctor apart couple dull...
4,Oh dear. Some of the best talent in British TV...,0,oh dear best talent british tv make serial ass...
...,...,...,...
2495,Home Room really surprised me. In comparison t...,1,home room really surprise comparison movie wri...
2496,Too bad somebody did not have the smarts to re...,1,bad somebody smart release movie theater never...
2497,I know this movie is a low-budget horror movie...,0,know movie lowbudget horror movie intend favor...
2498,Still funny after all these years. Midnight Ma...,1,still funny year midnight madness good enterta...


In [ ]:
# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(temp['Processed_Reviews'])
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

list_tokenized_train = tokenizer.texts_to_sequences(temp['Processed_Reviews'])
  
RNN_CELL_SIZE = 32
  
MAX_LEN = 130  
X_train = pad_sequences(list_tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train = temp['sentiment']



In [ ]:
X_train, X_test = train_test_split(X_train, test_size=0.2,  random_state = 42)
y_train, y_test = train_test_split(y_train, test_size=0.2,  random_state = 42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(2000, 130)
(500, 130)
(2000,)
(500,)


In [ ]:
def create_embedding_matrix(word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    with open('/content/drive/MyDrive/Colab Notebooks/Dataset/cc.en.300.vec') as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)


In [ ]:
from tensorflow.keras import regularizers
import time
LOG_DIR = f"{int(time.time())}"
seed_value= 0



def build_model(hp):
  
  reproduceResult()

  print('Ya it comes here')
  unit_attention = hp.Int("attention_unit",min_value =16, max_value = 128, step = 16)
  cnn_1_unit = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn_1_dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_unit = hp.Int("lstm_unit",min_value =32, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)
  cnn_2_unit = hp.Int("cnn_2_unit",min_value =32, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn_2_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)



  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,
                          weights = [embedd_matrix])(seq_input)

  cnn = keras.layers.Conv1D(cnn_1_unit,3)(embedded)
  cnn = keras.layers.Activation(activation='relu')(cnn)
  cnn = keras.layers.BatchNormalization()(cnn)
  cnn = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn)

  attention_vec = keras.layers.TimeDistributed(keras.layers.Dense(unit_attention))(cnn)
  attention_vec = keras.layers.Reshape((128,unit_attention))(attention_vec)
  attention_vec = keras.layers.Activation('relu', name = 'cnn_attention_vec')(attention_vec)
  attention_output = keras.layers.Dot(axes = 1)([cnn, attention_vec])


  lstm = keras.layers.Bidirectional(keras.layers.LSTM(lstm_unit, 
                                                      return_sequences=True,input_shape =(128,)))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)
  
  

  cnn_2 = keras.layers.Conv1D(cnn_2_unit,3)(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_2)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_2)
  output = keras.layers.Dense(2, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

  return model

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')

# clr_step_size = int((len(X_train)/32))
# base_lr = 2e-5
# max_lr = 1e-3
# mode = 'exp_range'


# clr = CyclicLR(base_lr = base_lr, max_lr = max_lr, step_size = clr_step_size, mode = mode)

tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 30,
    executions_per_trial = 1,
    directory = LOG_DIR
    )
  
tuner.search(x=X_train,y = y_train,epochs = 20, batch_size = 50, validation_data = (X_test,y_test),callbacks = [stop_early])


tuner.results_summary()

Trial 30 Complete [00h 03m 10s]
val_accuracy: 0.8159999847412109

Best val_accuracy So Far: 0.8479999899864197
Total elapsed time: 02h 43m 13s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in 1632298135/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
attention_unit: 64
cnn_1_unit: 80
cnn_1_dropout: 0.2
lstm_unit: 160
lstm_dropout: 0.1
cnn_2_unit: 32
cnn_2_dropout: 0.2
Score: 0.8479999899864197
Trial summary
Hyperparameters:
attention_unit: 96
cnn_1_unit: 32
cnn_1_dropout: 0.2
lstm_unit: 160
lstm_dropout: 0.2
cnn_2_unit: 128
cnn_2_dropout: 0.4
Score: 0.8379999995231628
Trial summary
Hyperparameters:
attention_unit: 112
cnn_1_unit: 32
cnn_1_dropout: 0.2
lstm_unit: 160
lstm_dropout: 0.2
cnn_2_unit: 128
cnn_2_dropout: 0.5
Score: 0.8360000252723694
Trial summary
Hyperparameters:
attention_unit: 112
cnn_1_unit: 48
cnn_1_dropout: 0.1
lstm_unit: 128
lstm_dropout: 0.5
cnn_2_unit: 128
cnn_2_dropout: 0.4
Score: